In [ ]:
import os
import sys
import numpy as np
from numpy.linalg import inv
from numpy.random import RandomState, SeedSequence, MT19937
from scipy.interpolate import interp1d
from scikits.odes import dae

if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from pfcommon import OU, parse_sparse_matrix_file
from filter_OU_inputs import run_welch
import daes

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size': 3})
matplotlib.rc('ytick.major', **{'width': lw, 'size': 3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size': 1.5})

FIGURES_DIR = 'figures'

#### Base units
Base apparent power and frequency as defined in PowerFactory. Base torque is computed accordingly.

In [ ]:
S_base = 1e6                # [VA] base apparent power
V_base = 10e3 / np.sqrt(3)  # [V] base voltage
I_base = S_base / (3 * V_base)
ω_base = 2 * np.pi * 50     # [Hz] base frequency
T_base = S_base / ω_base    # [Nm] base torque
Z_base = V_base / I_base
Y_base = 1 / Z_base         # [S] base admittance
L_base = Z_base / ω_base    # [H] base inductance
ψ_base = L_base * I_base    # [Wb/turns] base flux
print('S_base = {:g} MVA'.format(S_base * 1e-6))
print('V_base = {:g} kV'.format(V_base * 1e-3))
print('I_base = {:g} A'.format(I_base))
print('F_base = {:g} Hz'.format(ω_base / (2 * np.pi)))
print('T_base = {:g} Nm'.format(T_base))
print('Z_base = {:g} Ω'.format(Z_base))
print('Y_base = {:g} S'.format(Y_base))
print('L_base = {:g} H'.format(L_base))
print('ψ_base = {:g} Wb/turn'.format(ψ_base))

Load the power flow data:

In [ ]:
data_folder = '../data/SM_with_load/adynamic_load_const_S/LD1'
AC_fname = os.path.join(data_folder, 'SM_with_load_AC.npz')
data = np.load(AC_fname, allow_pickle=True)
PF = data['PF_without_slack'].item()
PF_load = PF['loads']['LD1']
PF_gen = PF['SMs']['G1']
PF_bus = PF['buses']['Bus1']

## Calculate all system parameters

#### Bus parameters

In [ ]:
assert abs(V_base - PF_bus['V'] * 1e3 / PF_bus['u']) < 1e-10
uBr = PF_bus['ur'] * V_base
uBi = PF_bus['ui'] * V_base
assert abs(PF_bus['V'] * 1e3 - np.abs(uBr + 1j * uBi)) < 1e-10
uB = uBr + 1j * uBi
print('======== BUS ========')
print('V_bus = {:.2f} kV'.format((uBr + 1j * uBi) * 1e-3))

#### Load parameters

In [ ]:
assert abs(V_base - PF_load['V'] * 1e3 / PF_load['u']) < 1e-10
P_load = PF_load['P'] * 1e6   # [W]
Q_load = PF_load['Q'] * 1e6   # [VAR]
S_load = P_load + 1j * Q_load # [VA]
V_load = (PF_load['ur'] + 1j * PF_load['ui']) * V_base
I_base_load = np.abs(S_load) / (3 * V_base)
I_load = (S_load / (3 * V_load)).conjugate()
Z_load = V_load / I_load
iLr, iLi = float(I_load.real), float(I_load.imag)
assert abs(I_base_load - np.abs(I_load)) < 1e-10, 'Base load current does not match actual load current'
assert abs(PF_load['ir'] * np.abs(I_load) - iLr) < 1e-3, 'Real part of the current does not match'
assert abs(PF_load['ii'] * np.abs(I_load) - iLi) < 1e-3, 'Imaginary part of the current does not match'
iL = iLr + 1j * iLi
print('========== LOAD ========')
print('     S_load = {:g} MVA'.format(S_load * 1e-6))
print('I_base_load = {:.2f} A'.format(I_base_load))
print('     I_load = {:.2f} A'.format(I_load))
print('     Z_load = {:.2f} Ω'.format(Z_load))

#### Generator parameters

In [ ]:
# with the following value of generator apparent power, the Z_base of the
# generator is equivalent to the system one (i.e., the one computed above
# using V_base and S_base)
S_gen = 1e6                   # [MVA] apparent power of the generator
tag = 8                       # [s] acceleration time constant
H = tag / 2                   # [s] inertia constant
E_kin = H * S_gen             # [J] kinetic energy of the generator
J = 2 * E_kin / (ω_base ** 2) # [kgm2] moment of inertia of the generator

I_base_gen = S_gen / (3 * V_base) # [A]
iGr = PF_gen['ir'] * I_base_gen   # [A]
iGi = PF_gen['ii'] * I_base_gen   # [A]
iG = iGr + 1j * iGi               # [A]
Z_base_gen = V_base / I_base_gen  # [Ω]
assert abs(PF_gen['I'] * 1e3 / PF_gen['i'] - I_base_gen) < 1e-4
assert abs(PF_gen['I'] * 1e3 - np.abs(iGr + 1j * iGi)) < 1e-4

rstr, xstr = 0.2, 0.4                         # [pu] stator parameters
R_gen = rstr * Z_base_gen                     # [Ω]
X_gen = xstr * Z_base_gen                     # [Ω]
Z_gen = R_gen + 1j * X_gen                    # [Ω]

# the voltage of the ideal generator in the synchronous machine
ve = uB + Z_gen * iG
E0, ϕG = float(np.abs(ve)), float(np.angle(ve))

print('============ SM ===========')
print('       tag = {:g} s'.format(tag))
print('         H = {:g} s'.format(H))
print('     E_kin = {:g} MJ'.format(E_kin*1e-6))
print('         J = {:g} kgm2'.format(J))
print('I_base_gen = {:.2f} A'.format(I_base_gen))
print('Z_base_gen = {:.2f} Ω'.format(Z_base_gen))
print('     I_gen = {:.2f} A'.format(iGr + 1j * iGi))
print('     Z_gen = {:.2f} Ω'.format(Z_gen))
print('        E0 = {:.2f} kV'.format(E0 * 1e-3))
print('         ϕ = {:.2f} deg'.format(np.rad2deg(ϕG)))

#### Parameters of the leak conductance connected to the bus

In [ ]:
Ggnd_pu = 11
Ggnd = Ggnd_pu * 1e-9 * S_base / (3 * V_base ** 2)
Ygnd = (iG - iL) / uB   # [S]
assert abs(Ygnd.real - Ggnd) < 1e-12
print('Ggnd = {:.2f} pS'.format(Ggnd * 1e12))

### Torques
General parameters:

In [ ]:
n = 1     # [pu]
nref = 1  # [pu]
cosn = 1  # rated power factor

#### Electrical torque

In [ ]:
ut = PF_gen['ur'] + 1j * PF_gen['ui']                    # [pu]
it = PF_gen['ir'] + 1j * PF_gen['ii']                    # [pu]
ψstr = (ut + rstr * it) / (1j * n)                       # [pu]
te = (it.imag * ψstr.real - it.real * ψstr.imag) / cosn  # [pu]
Te = te * T_base                                         # [Nm]
print('Stator flux: {:g} p.u.'.format(ψstr))
print('Electrical torque: {:g} p.u.'.format(te))
print('Electrical torque: {:g} Nm'.format(Te))

#### Mechanical torque

In [ ]:
dpu   = 0
xmdm  = 0
addmt = 0
pt = (te + dpu * n + xmdm) * n
tm = pt / n - (xmdm + dpu * n - addmt)
Tm = tm * T_base
print('Mechanical torque: {:g} p.u.'.format(tm))
print('Mechanical torque: {:g} Nm'.format(Tm))

#### Damping torques (not used for now)

In [ ]:
if False:
    dkd, dpe = 0, 0
    tdkd = dkd * (n - nref)
    tdpe = dpe / n * (n - nref)
    Tdkd, Tdpe = tdkd * T_base, tdpe * T_base
    print('Damping torques: ({:g},{:g}) p.u.'.format(tdkd, tdpe))
    print('Damping torques: ({:g},{:g}) Nm.'.format(Tdkd, Tdpe))

## Time domain simulations

### Data from transient simulation performed in PowerFactory

In [ ]:
PF_filename = os.path.join(data_folder, 'SM_with_load_tran_0.01.npz')
PF_data = np.load(PF_filename, allow_pickle=True)
t_tran_PF = PF_data['time']
dt_tran_PF = t_tran_PF[1] - t_tran_PF[0]
ou_PF = PF_data['OU_P'].squeeze()
t_ou_PF = np.arange(ou_PF.size) * dt_tran_PF
tran_data = PF_data['data'].item()

ω_tran_PF = tran_data['gen']['s:speed'] * ω_base
ϕ_tran_PF = tran_data['gen']['s:phi']
ur_tran_PF, ui_tran_PF = tran_data['bus']['m:ur'] * V_base, tran_data['bus']['m:ui'] * V_base
ir_load_tran_PF, ii_load_tran_PF = tran_data['load']['m:ir:bus1'] * I_base_load, tran_data['load']['m:ii:bus1'] * I_base_load
ir_gen_tran_PF, ii_gen_tran_PF = tran_data['gen']['m:ir:bus1'] * I_base_gen, tran_data['gen']['m:ii:bus1'] * I_base_gen

P_tran_PF = []
abs_tran_PF = []
idx = t_tran_PF > 100
for y_tran_PF in [ω_tran_PF, ϕ_tran_PF, ur_tran_PF, ui_tran_PF, \
                  ir_load_tran_PF, ii_load_tran_PF, ir_gen_tran_PF, ii_gen_tran_PF]:
    Δy_tran_PF = y_tran_PF[idx] - y_tran_PF[idx].mean()
    ret = run_welch(Δy_tran_PF, dt_tran_PF, window=200/dt_tran_PF, onesided=True)
    freq_tran_PF = ret[0]
    P_tran_PF.append(ret[1])
    abs_tran_PF.append(ret[2])
P_tran_PF = np.array(P_tran_PF)
abs_tran_PF = np.array(abs_tran_PF)

First make sure that the PF solution is indeed an equilibrium for the system:

In [ ]:
dkd, dpe = 0., 0.
sys = daes.SynchMachAdynLoadConstS(w_base=ω_base, V_base=V_base, P_load=P_load,
                                   Q_load=Q_load, Ggnd=Ggnd, rstr=rstr, R_gen=R_gen, X_gen=X_gen,
                                   tag=tag, E0=E0, phiG=ϕG, cosn=cosn, I_base_gen=I_base_gen,
                                   pt=pt, xmdm=xmdm, dpu=dpu, addmt=addmt, dkd=dkd, dpe=dpe, w_ref=ω_base)
res_fn = daes.resindex1()
jac_fn = daes.jacindex1()
res_fn.set_power_system(sys)
jac_fn.set_power_system(sys)
y0 = np.array([ω_base, ϕG, uBr, uBi, iLr, iLi, iGr, iGi], dtype=float)
N_vars = y0.size
ydot0 = np.zeros(N_vars, dtype=float)
res0 = np.zeros(N_vars, dtype=float)
res_fn.evaluate(0, y0, ydot0, res0, None)
assert np.allclose(res0, np.zeros(N_vars), rtol=1e-6, atol=1e-6)

Then integrate the system:

In [ ]:
dt = 5e-3
t0, t1 = 0, 300
force = False
outfile = 'SM_with_load_tstop_{}_dt_{:g}.npz'.format(t1, dt)
if force or not os.path.isfile(outfile):
    t_eval = np.r_[t0 : t1 : dt]
    try:
        seed = int(PF_data['OU_seeds'][0])
    except:
        seed = 1983
    print(f'Seed: {seed}.')
    rs = RandomState(MT19937(SeedSequence(seed)))
    μ = P_load
    dP = 0.01
    σ = dP * P_load
    τ = 20e-3
    ou = OU(dt, μ, σ, τ, t_eval.size, rs)
    sys.OU = np.c_[t_eval, ou]
    solver = dae('ida',
                 res_fn, 
                 compute_initcond='yp0',
                 first_step_size=1e-12,
                 atol=1e-6,
                 rtol=1e-6,
                 max_steps=1000,
                 # min_step_size=dt/100,
                 max_step_size=dt,
                 # jacfn=jac_fn,
                 algebraic_vars_idx=sys.algebraic_vars_idx,
                 compute_initcond_t0=10,
                 exclude_algvar_from_error=sys.exclalg_err,
                 old_api=False)
    sol = solver.solve(t_eval, y0, ydot0)
    t_tran, y_tran = sol.values.t, sol.values.y
    np.savez_compressed(outfile, dt=dt, t0=t0, t1=t1, P_load=P_load,
                    OU_seed=seed, OU_mean=μ, OU_stddev=σ, OU_tau=τ,
                    t=t_tran, y=y_tran, ydot=sol.values.ydot)
else:
    tran = np.load(outfile, allow_pickle=True)
    t_tran = tran['t']
    y_tran = tran['y']
    μ, σ, τ = tran['OU_mean'], tran['OU_stddev'], tran['OU_tau']
    seed = tran['OU_seed'].item()
    ou = OU(dt, μ, σ, τ, t_tran.size, RandomState(MT19937(SeedSequence(seed))))

In [ ]:
var_names = r'$\omega_{gen}$', r'$\phi$', r'$v_{bus}^r$', r'$v_{bus}^i$', \
    r'$i_{load}^r$', r'$i_{load}^i$', r'$i_{gen}^r$', r'$i_{gen}^i$'
coeffs = [1/(2*np.pi), 1e-3, 1e-3, 1, 1, 1, 1, 1]
units = ['Hz', 'rad', 'kV', 'kV', 'A', 'A', 'A', 'A']
rows, cols = 4, 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*4, rows*2), sharex=True, sharey=False, squeeze=False)
for k in range(N_vars):
    i,j = k // cols, k % cols
    a = ax[i,j]
    a.plot(t_tran, y_tran[:,k] * coeffs[k], 'k', lw=1)
    a.set_ylabel('{} [{}]'.format(var_names[k], units[k]), fontsize=10)
for a in ax[-1,:]:
    a.set_xlabel('Time [s]')
sns.despine()
fig.tight_layout()

... and compute the PSD of the solution:

In [ ]:
P_tran, abs_tran = [], []
for output_var_idx in range(N_vars):
    Δy_tran = y_tran[:,output_var_idx] - y_tran[:,output_var_idx].mean()
    ret = run_welch(Δy_tran, dt, window=200/dt, onesided=True)
    freq_tran = ret[0]
    P_tran.append(ret[1])
    abs_tran.append(ret[2])
P_tran = np.array(P_tran)
abs_tran = np.array(abs_tran)

Check that we get the same results as PowerFactory

In [ ]:
tstop = 60
fig,ax = plt.subplots(2, 1, figsize=(5,5), sharex=True)
idx = t_tran_PF < tstop
ax[0].plot(t_tran_PF[idx], ω_tran_PF[idx] / (2 * np.pi), 'r', lw=1, label='PF')
idx = t_ou_PF < tstop
ax[1].plot(t_ou_PF[idx], ou_PF[idx], 'r', lw=1, label='PF')
idx = t_tran < tstop
ax[0].plot(t_tran[idx], y_tran[idx,0] / (2 * np.pi), 'k', lw=1, label='Python')
ax[1].plot(t_tran[idx], ou[idx] * 1e-6, 'k', lw=1, label='Python')
ax[-1].set_xlabel('Time [s]')
ax[0].set_ylabel('F [Hz]')
ax[1].set_ylabel('P [MW]')
ax[0].legend(loc='lower left', frameon=False, fontsize=fontsize-1)
sns.despine()
fig.tight_layout()

## Building the transfer functions

First compute the Jacobian matrix:

In [ ]:
J1 = np.zeros((N_vars, N_vars))
err = jac_fn.evaluate(0, y0, ydot0, res0, 1, J1, None)
J1 *= -1

Alternatively, the Jacobian matrix can be computed numerically:

In [ ]:
res = np.zeros(N_vars, dtype=float)
J2 = np.zeros((N_vars, N_vars))
for j in range(N_vars):
    dy = max(1e-3 * np.abs(y0[j]), 1e-3)
    y = y0 + np.array([dy if i == j else 0 for i in range(N_vars)])
    res_fn.evaluate(0, y, ydot0, res, None)
    J2[:,j] = (res - res0) / dy
J2 *= -1

In [ ]:
J1[-2:, :] *= -1
J2[-2:, :] *= -1
print_row = lambda A, row, fmt='{:13g} ' * N_vars: print(('[{:2d}] ' + fmt).format(row+1, *A[row,:]))
for row in range(N_vars):
    print_row(J1, row)
    # print_row(J2, row)
    # print()

In [ ]:
analytic_jacobian = True
J = J1 if analytic_jacobian else J2

#### Compare the Jacobian calculated above with the one calculated by PowerFactory:

In [ ]:
J_PF = parse_sparse_matrix_file(os.path.join(data_folder, 'Jacobian.mtl'))[:N_vars, :N_vars]
for row in range(N_vars):
    print_row(J_PF, row)
# J = J_PF

In [ ]:
def compute_matrices(J, N_state_vars, N_algebraic_vars):
    Jfx = J[:N_state_vars, :N_state_vars]
    Jfy = J[:N_state_vars, N_state_vars:]
    Jgx = J[N_state_vars:, :N_state_vars]
    Jgy = J[N_state_vars:, N_state_vars:]
    Jgy_inv = inv(Jgy)
    A = Jfx - np.dot(np.dot(Jfy, Jgy_inv), Jgx)
    B = - np.dot(Jfy, Jgy_inv)
    C = - np.dot(Jgy_inv, Jgx)
    D = - Jgy_inv
    return A,B,C,D

N_state_vars = 2
N_algebraic_vars = N_vars - N_state_vars
A,B,C,D = compute_matrices(J, N_state_vars, N_algebraic_vars)
A_PF,_,_,_ = compute_matrices(J, N_state_vars, N_algebraic_vars)
print('             Eigenvalues: ', np.linalg.eig(A)[0])
print('PowerFactory eigenvalues: ', np.linalg.eig(A_PF)[0])

In [ ]:
Fmin, Fmax = -6, 2
steps_per_decade = 100
F = np.logspace(Fmin, Fmax, (Fmax - Fmin) * steps_per_decade + 1)
N_input_loads = 1
TF = np.zeros((F.size, N_input_loads, N_vars), dtype=complex)
OUT = np.zeros((F.size, N_input_loads, N_vars), dtype=complex)

In [ ]:
coeffs = np.zeros(N_algebraic_vars)
coeffs[2] = uB.real / np.abs(uB) ** 2
coeffs[3] = uB.imag / np.abs(uB) ** 2

In [ ]:
I = np.eye(N_state_vars)
α = 1 / τ
c = σ * np.sqrt(2 / τ)
j = 0
for i, f in enumerate(F):
    M = 1j * 2 * np.pi * f * I - A # sI - A
    MINVxB = np.dot(inv(M), B)            # (sI - A)^-1 x B
    psd = np.sqrt((c / α)**2 / (1 + (2 * np.pi * F[i] / α)**2)) / 3
    TF[i, j, :N_state_vars] = np.dot(MINVxB, coeffs)
    TF[i, j, N_state_vars:] = np.dot(np.dot(C, MINVxB) + D, coeffs)
    OUT[i, j, :N_state_vars] = np.dot(MINVxB, psd * coeffs)
    OUT[i, j, N_state_vars:] = np.dot(np.dot(C, MINVxB) + D, psd * coeffs)

Compare the results:

In [ ]:
input_var_idx = 0
dB = 10
rows, cols = 4, 2
fig,ax = plt.subplots(rows, cols, figsize=(cols*4, rows*2), sharex=True, sharey=False, squeeze=False)
for output_var_idx in range(N_vars):
    i,j = output_var_idx // cols, output_var_idx % cols
    a = ax[i,j]
    Y = OUT[:, input_var_idx, output_var_idx]
    if np.all(Y == 0 + 1j*0):
        a.set_visible(False)
        continue
    a.plot(freq_tran, dB * np.log10(abs_tran[output_var_idx]) if dB in (10, 20) else abs_tran,
           'k', lw=1, label='Python tran')
    a.plot(freq_tran_PF, dB * np.log10(abs_tran_PF[output_var_idx]) if dB in (10, 20) else abs_tran_PF,
           'tab:green', lw=1, label='PF tran')
    y = dB * np.log10(np.abs(Y)) if dB in (10, 20) else np.abs(Y)
    a.semilogx(F, y, 'r', lw=1, label='Theory')
    a.set_title(var_names[output_var_idx], fontsize=10)
ax[0,0].set_xlim([1e-3, 1e2])
ax[0,0].legend(loc='lower left', frameon=False, fontsize=fontsize-1)
for a in ax[-1,:]:
    a.set_xlabel('Frequency [Hz]')
for a in ax[:,0]:
    a.set_ylabel(r'|Y(j$\omega$)|')
sns.despine()
fig.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, os.path.splitext(outfile)[0] + '.pdf'))